In [1]:
import os,sys
sys.path.append("%s/analysis-CommonTools/"%os.getenv('HOME'))
from common_imports import *
from matplotlib_tools import *

In [2]:
import ROOT

Welcome to JupyROOT 6.26/00


In [3]:
ROOT.gStyle.SetOptStat(0)

In [4]:
%jsroot on

In [5]:
do_iev_check = False

lep_accept_pt = 0.4 #GeV
lep_accept_angle = radians(40)

hadron_accept_angle = radians(40)
hadron_accept_ke = 0.060 #GeV

In [6]:
c = ROOT.TCanvas()

In [8]:
dir_files_GEM21_11b = "/Users/wketchum/Data/LDMX/eN_Ti_GENIE_v3_2_0/ldmx_eTi_4GeV_GEM21_11b_00"
dir_files_G18_02a = "/Users/wketchum/Data/LDMX/eN_Ti_GENIE_v3_2_0/ldmx_eTi_4GeV_G18_02a_00"

In [9]:
gst_files_GEM21_11b = glob.glob(f"{dir_files_GEM21_11b}/*gst.root")
gst_files_G18_02a = glob.glob(f"{dir_files_G18_02a}/*gst.root")

In [26]:
gst_chain_GEM21_11b = ROOT.TChain("gst")
for f in gst_files_GEM21_11b:
    gst_chain_GEM21_11b.Add(f)

In [27]:
df_gst_GEM21_11b_all = ROOT.RDataFrame(gst_chain_GEM21_11b)

In [28]:
gst_chain_G18_02a = ROOT.TChain("gst")
for f in gst_files_G18_02a:
    gst_chain_G18_02a.Add(f)

In [29]:
df_gst_G18_02a_all = ROOT.RDataFrame(gst_chain_G18_02a)

In [30]:
def define_df_gst(df_gst_all):
    df_gst_all = df_gst_all.Define("tmp","Ev")
    df_gst_all = df_gst_all.Define("ptl","sqrt(pxl*pxl+pyl*pyl)")
    df_gst_all = df_gst_all.Define("thetazl","atan2(ptl,pzl)")
    
    df_gst_all = df_gst_all.Define("energy_transfer","Ev-El")

    hadron_vars = ["pdg","E","px","py","pz","pt","p","mass","ke","thetaxz","thetayz","thetaz"]
    for s in ["i","f"]:
        df_gst_all = df_gst_all.Define(f"thetaxz{s}",f"atan2(px{s},pz{s})")
        df_gst_all = df_gst_all.Define(f"thetayz{s}",f"atan2(py{s},pz{s})")
        df_gst_all = df_gst_all.Define(f"pt{s}",f"sqrt(px{s}*px{s}+py{s}*py{s})")
        if s=="i":
            df_gst_all = df_gst_all.Define(f"p{s}",f"sqrt(px{s}*px{s}+py{s}+py{s}+pz{s}*pz{s})")
        df_gst_all = df_gst_all.Define(f"thetaz{s}",f"atan2(pt{s},pz{s})")
        df_gst_all = df_gst_all.Define(f"mass{s}",f"sqrt(E{s}*E{s}-p{s}*p{s})")
        df_gst_all = df_gst_all.Define(f"ke{s}",f"E{s}-mass{s}")
    
        hadronx_vars = [ f"{hv}{s}" for hv in hadron_vars ]
        
        for var in hadronx_vars:
            df_gst_all = df_gst_all.Define(f"{var}_p",f"{var}[pdg{s}==2212]")
            df_gst_all = df_gst_all.Define(f"{var}_n",f"{var}[pdg{s}==2112]")
            df_gst_all = df_gst_all.Define(f"{var}_pi0",f"{var}[pdg{s}==111]")
            df_gst_all = df_gst_all.Define(f"{var}_piplus",f"{var}[pdg{s}==211]")
            df_gst_all = df_gst_all.Define(f"{var}_piminus",f"{var}[pdg{s}==-211]")
            
            #if s=="i": 
            #    continue
            
            df_gst_all = df_gst_all.Define(f"{var}_p_accept",f"{var}[pdg{s}==2212 && thetaz{s}<{hadron_accept_angle} && ke{s}>{hadron_accept_ke}]")
            df_gst_all = df_gst_all.Define(f"{var}_n_accept",f"{var}[pdg{s}==2112 && thetaz{s}<{hadron_accept_angle} && ke{s}>{hadron_accept_ke}]")
            #df_gst_all = df_gst_all.Define(f"{var}_pi0_accept",f"{var}_pi0[pdg{s}==111 && thetaz{s}<{hadron_accept_angle} && ke{s}>{hadron_accept_ke}]")
            df_gst_all = df_gst_all.Define(f"{var}_piplus_accept",f"{var}[pdg{s}==211 && thetaz{s}<{hadron_accept_angle} && ke{s}>{hadron_accept_ke}]")
            df_gst_all = df_gst_all.Define(f"{var}_piminus_accept",f"{var}[pdg{s}==-211 && thetaz{s}<{hadron_accept_angle} && ke{s}>{hadron_accept_ke}]")
    
    return df_gst_all


In [31]:
df_gst_GEM21_11b_all = define_df_gst(df_gst_GEM21_11b_all)
df_gst_G18_02a_all = define_df_gst(df_gst_G18_02a_all)

In [32]:
df_gst_GEM21_11b_accept = df_gst_GEM21_11b_all.Filter(f"ptl>0.4 && abs(thetazl)<{radians(40)}")
df_gst_G18_02a_accept = df_gst_G18_02a_all.Filter(f"ptl>0.4 && abs(thetazl)<{radians(40)}")

In [33]:
h_kef_p_accept_GEM21_11b = df_gst_GEM21_11b_accept.Histo1D(("","",1000,0,3.0),"kef_p_accept")
h_kef_p_accept_G18_02a = df_gst_G18_02a_accept.Histo1D(("","",1000,0,3.0),"kef_p_accept")

In [34]:
h_kef_p_accept_GEM21_11b.SetLineColor(ROOT.kRed)
h_kef_p_accept_GEM21_11b.Draw()
h_kef_p_accept_G18_02a.Draw("same")
c.Draw()

In [ ]:
h3 = df_gst_lep_accept.Histo1D(("","",1000,0,3.0),"ptf_piplus")

In [ ]:
h3.Draw()
c.Draw()

In [ ]:
h4 = df_gst_lep_accept.Histo1D(("","",1000,0,3.0),"pf_piplus")
h4.Draw()
c.Draw()

In [ ]:
h4.SetTitle("#pi^{+};Momentum (GeV/c);")
h4.Draw()
c.Draw()

In [ ]:
h_n_p = df_gst_lep_accept.Define("n_p_accept","Ef_p_accept.size()").Histo1D(("","",10,-0.5,9.5),"n_p_accept")

In [ ]:
h_n_p.Draw()
c.Draw()

In [ ]:
h_kef_p_max = df_gst_GEM21_11b_accept.Define("kef_p_max","Max(kef_p)").Histo1D(("","",1000,0,3.0),"kef_p_max")
h_kei_p_max = df_gst_GEM21_11b_accept.Define("kei_p_max","Max(kei_p)").Histo1D(("","",1000,0,3.0),"kei_p_max")

In [ ]:
h_kef_p_max.SetLineColor(ROOT.kRed)
h_kef_p_max.Draw()
h_kei_p_max.Draw("same")
c.Draw()

In [ ]:
h_kef_piplus_max = df_gst_GEM21_11b_accept.Define("kef_piplus_max","Max(kef_piplus)").Histo1D(("","",1000,0,3.0),"kef_piplus_max")
h_kei_piplus_max = df_gst_GEM21_11b_accept.Define("kei_piplus_max","Max(kei_piplus)").Histo1D(("","",1000,0,3.0),"kei_piplus_max")

In [ ]:
h_kef_piplus_max.SetLineColor(ROOT.kRed)
h_kef_piplus_max.Draw()
h_kei_piplus_max.Draw("same")
c.Draw()

In [ ]:
npy_df_gst_GEM21_11b_accept = df_gst_GEM21_11b_accept.AsNumpy()

In [ ]:
npy_df_ke_p = df_gst_GEM21_11b_accept.AsNumpy(columns=["kef_p","kei_p","kef_p_accept"])

In [ ]:
np.shape(npy_df_ke_p['kef_p'])